In [0]:
def replace(col):
    import re
    return re.sub("[^0-9a-zA-Z]+", "_", col)
from functools import reduce
from pyspark.sql import DataFrame

In [0]:
dfs = [] 
for i in range(1,6):
    path = "/FileStore/tables/Epsilon_1_"+str(i)+".csv"
    print(path)
    tmp = spark.read.csv(path, header="true", inferSchema="true")
    new_c = [replace(col) for col in tmp.columns]
    tmp = tmp.toDF(*new_c)
    dfs.append(tmp)

/FileStore/tables/Epsilon_1_1.csv


In [0]:
df = reduce(DataFrame.unionByName, dfs)
del(dfs)

In [0]:
df.count()

Out[10]: 8930770

In [0]:
#save

#df.write.format("delta").saveAsTable("bronze_epsilon_1")
df.write.parquet("/FileStore/tables/bronze_1")

In [0]:
dfs2 = [] 
rows = []
for i in range(1,33):
    path = "/FileStore/tables/Epsilon_2_"+str(i)+".csv"
    tmp = spark.read.csv(path, header="true", inferSchema="true", sep='|')
    new_c = [replace(col) for col in tmp.columns]
    tmp = tmp.toDF(*new_c)
    rows.append(tmp.count())
    dfs2.append(tmp)
df2 = reduce(DataFrame.unionByName, dfs2)
print(df2.count())
print(sum(rows))
del(dfs2)

4628137
4628137


In [0]:
#save
#dbutils.fs.rm("dbfs:/user/hive/warehouse/bronze_epsilon_2/", True)
#df2.write.format("delta").saveAsTable("bronze_epsilon_2")
df2.write.parquet("/FileStore/tables/df2")
del(df2)

In [0]:
#reload
df2 = spark.read.parquet("/FileStore/tables/df2")

In [0]:
len(df2.columns)

Out[2]: 1291